In [ ]:
# default_exp utils.duration

In [ ]:
# export
import torch
import torch.nn as nn
from uberduck_ml_dev.models.common import LinearNorm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn import functional as F
import numpy as np
import math

import pdb


class PositionalEncoding(nn.Module):
    """
        sinusoidal positional encoding
        https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return self.pe[x]


class GaussianUpsampling(nn.Module):
    """
        Non-attention Tacotron:
            - https://arxiv.org/abs/2010.04301
        this source code is implemenation of the ExpressiveTacotron from BridgetteSong
            - https://github.com/BridgetteSong/ExpressiveTacotron/blob/master/model_duration.py
    """

    def __init__(self, hparams):
        super(GaussianUpsampling, self).__init__()
        self.mask_score = -1e15

    def forward(self, encoder_outputs, durations, vars, input_lengths=None):
        """ Gaussian upsampling
        PARAMS
        ------
        encoder_outputs: Encoder outputs  [B, N, H]
        durations: phoneme durations  [B, N]
        vars : phoneme attended ranges [B, N]
        input_lengths : [B]
        RETURNS
        -------
        encoder_upsampling_outputs: upsampled encoder_output  [B, T, H]
        """
        B = encoder_outputs.size(0)
        N = encoder_outputs.size(1)
        T = int(torch.sum(durations, dim=1).max().item())
        c = torch.cumsum(durations, dim=1).float() - 0.5 * durations
        c = c.unsqueeze(2)  # [B, N, 1]
        t = (
            torch.arange(T, device=encoder_outputs.device).expand(B, N, T).float()
        )  # [B, N, T]
        vars = vars.view(B, -1, 1)  # [B, N, 1]

        w_t = -0.5 * (
            np.log(2.0 * np.pi) + torch.log(vars) + torch.pow(t - c, 2) / vars
        )  # [B, N, T]

        if input_lengths is not None:
            input_masks = ~self.get_mask_from_lengths(input_lengths, N)  # [B, N]
            input_masks = torch.tensor(input_masks, dtype=torch.bool, device=w_t.device)
            masks = input_masks.unsqueeze(2)
            w_t.data.masked_fill_(masks, self.mask_score)
        w_t = F.softmax(w_t, dim=1)

        encoder_upsampling_outputs = torch.bmm(
            w_t.transpose(1, 2), encoder_outputs
        )  # [B, T, encoder_hidden_size]

        return encoder_upsampling_outputs

    def get_mask_from_lengths(self, lengths, max_len=None):
        if max_len is None:
            max_len = max(lengths)
        ids = torch.arange(0, max_len, device=lengths.device)
        mask = ids < lengths.reshape(-1, 1)
        return mask


class DurationPredictor(nn.Module):
    """Duration Predictor module:
        - two stack of BiLSTM
        - one projection layer
    """

    def __init__(self, hparams):
        super(DurationPredictor, self).__init__()
        assert (
            hparams.duration_lstm_dim % 2 == 0
        ), "duration_lstm_dim must be even [{}]".format(hparams.duration_lstm_dim)

        self.lstm = nn.LSTM(
            hparams.encoder_embedding_dim,
            int(hparams.duration_lstm_dim / 2),
            2,
            batch_first=True,
            bidirectional=True,
        )

        self.proj = LinearNorm(hparams.duration_lstm_dim, 1)
        self.relu = nn.ReLU()

    def forward(self, encoder_outputs, input_lengths=None):
        """
            :param encoder_outputs: [B, N, encoder_lstm_dim]
            :param input_lengths: [B, N]
            :return: [B, N]
        """

        B = encoder_outputs.size(0)
        N = encoder_outputs.size(1)

        ## remove pad activations
        if input_lengths is not None:
            encoder_outputs = pack_padded_sequence(
                encoder_outputs, input_lengths, batch_first=True, enforce_sorted=False
            )

        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(encoder_outputs)

        if input_lengths is not None:
            outputs, _ = pad_packed_sequence(outputs, batch_first=True)

        outputs = self.relu(self.proj(outputs))

        return outputs.view(B, N)


class RangePredictor(nn.Module):
    """Duration Predictor module:
        - two stack of BiLSTM
        - one projection layer
    """

    def __init__(self, hparams):
        super(RangePredictor, self).__init__()
        assert (
            hparams.range_lstm_dim % 2 == 0
        ), "range_lstm_dim must be even [{}]".format(hparams.range_lstm_dim)

        self.lstm = nn.LSTM(
            hparams.encoder_embedding_dim + 1,
            int(hparams.range_lstm_dim / 2),
            2,
            batch_first=True,
            bidirectional=True,
        )

        self.proj = LinearNorm(hparams.range_lstm_dim, 1)

    def forward(self, encoder_outputs, durations, input_lengths=None):
        """
            :param encoder_outputs:
            :param durations:
            :param input_lengths:
            :return:
        """

        concated_inputs = torch.cat([encoder_outputs, durations.unsqueeze(-1)], dim=-1)

        ## remove pad activations
        if input_lengths is not None:
            concated_inputs = pack_padded_sequence(
                concated_inputs, input_lengths, batch_first=True, enforce_sorted=False
            )

        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(concated_inputs)

        if input_lengths is not None:
            outputs, _ = pad_packed_sequence(outputs, batch_first=True)

        outputs = self.proj(outputs)
        outputs = F.softplus(outputs)
        return outputs.squeeze()


class PositionalEncoding(nn.Module):
    """
        sinusoidal positional encoding
        https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return self.pe[x]

In [ ]:
# export

import json


def forward_extractor(tokens, log_probs, blank):
    """Computes states f and p."""
    n, m = len(tokens), log_probs.shape[0]
    # `f[s, t]` -- max sum of log probs for `s` first codes
    # with `t` first timesteps with ending in `tokens[s]`.
    f = np.empty((n + 1, m + 1), dtype=float)
    f.fill(-(10 ** 9))
    p = np.empty((n + 1, m + 1), dtype=int)
    f[0, 0] = 0.0  # Start
    for s in range(1, n + 1):
        c = tokens[s - 1]
        for t in range((s + 1) // 2, m + 1):
            f[s, t] = log_probs[t - 1, c]
            # Option #1: prev char is equal to current one.
            if s == 1 or c == blank or c == tokens[s - 3]:
                options = f[s : (s - 2 if s > 1 else None) : -1, t - 1]
            else:  # Is not equal to current one.
                options = f[s : (s - 3 if s > 2 else None) : -1, t - 1]
            f[s, t] += np.max(options)
            p[s, t] = np.argmax(options)
    return f, p


def backward_extractor(f, p):
    """Computes durs from f and p."""
    n, m = f.shape
    n -= 1
    m -= 1
    durs = np.zeros(n, dtype=int)
    if f[-1, -1] >= f[-2, -1]:
        s, t = n, m
    else:
        s, t = n - 1, m
    while s > 0:
        durs[s - 1] += 1
        s -= p[s, t]
        t -= 1
    assert durs.shape[0] == n
    assert np.sum(durs) == m
    assert np.all(durs[1::2] > 0)
    return durs


def preprocess_tokens(tokens, blank):
    new_tokens = [blank]
    for c in tokens:
        new_tokens.extend([c, blank])
    tokens = new_tokens
    return tokens


# data_config = {
#     'manifest_filepath': "allfiles.json",
#     'sample_rate': 22050,
#     'labels': asr_model.decoder.vocabulary,
#     'batch_size': 1,
# }

# parser = nemo.collections.asr.data.audio_to_text.AudioToCharWithDursF0Dataset.make_vocab(
#     notation='phonemes', punct=True, spaces=True, stresses=False, add_blank_at="last"
# )

# dataset = nemo.collections.asr.data.audio_to_text._AudioTextDataset(
#     manifest_filepath=data_config['manifest_filepath'], sample_rate=data_config['sample_rate'], parser=parser,
# )

# dl = torch.utils.data.DataLoader(
#     dataset=dataset, batch_size=data_config['batch_size'], collate_fn=dataset.collate_fn, shuffle=False,
# )

# blank_id = asr_model.decoder.num_classes_with_blank - 1

# if os.path.exists(os.path.join(output_dir, "durations.pt")):
#     print("durations.pt already exists; skipping")
# else:
#     dur_data = {}
#     for sample_idx, test_sample in tqdm(enumerate(dl), total=len(dl)):
#         log_probs, _, greedy_predictions = asr_model(
#             input_signal=test_sample[0], input_signal_length=test_sample[1]
#         )

#         log_probs = log_probs[0].cpu().detach().numpy()
#         seq_ids = test_sample[2][0].cpu().detach().numpy()

#         target_tokens = preprocess_tokens(seq_ids, blank_id)

#         f, p = forward_extractor(target_tokens, log_probs, blank_id)
#         durs = backward_extractor(f, p)

#         dur_key = Path(dl.dataset.collection[sample_idx].audio_file).stem
#         dur_data[dur_key] = {
#             'blanks': torch.tensor(durs[::2], dtype=torch.long).cpu().detach(),
#             'tokens': torch.tensor(durs[1::2], dtype=torch.long).cpu().detach()
#         }

#         del test_sample

#     torch.save(dur_data, os.path.join(output_dir, "durations.pt"))

In [ ]:
# asr_model.decoder.vocabulary

[NeMo I 2022-01-10 20:00:35 cloud:56] Found existing object /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.5.1/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.
[NeMo I 2022-01-10 20:00:35 cloud:62] Re-using file from: /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.5.1/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo
[NeMo I 2022-01-10 20:00:35 common:728] Instantiating model from pre-trained checkpoint


[NeMo W 2022-01-10 20:00:35 features:243] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-01-10 20:00:35 features:265] PADDING: 1
[NeMo I 2022-01-10 20:00:35 features:275] STFT using conv
[NeMo I 2022-01-10 20:00:36 save_restore_connector:149] Model EncDecCTCModel was successfully restored from /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.5.1/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.
[NeMo I 2022-01-10 20:00:36 ctc_models:348] Changed decoder to output to ['_', '-', '!', "'", '(', ')', ',', '.', ':', ';', '?', ' ', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '@AA', '@AA0', '@AA1', '@AA2', '@AE', '@AE0', '@AE1', '@AE2', '@AH', '@AH0', '@AH1', '@AH2', '@AO', '@AO0', '@AO1', '@AO2', '@AW', '@AW0', '@AW1', '@AW2', '@AY', '@AY0', '@AY1', '@AY2', '@B', '@CH', '@D', '@DH', '@EH', '@EH0', '@EH1', '@EH2', '@ER',

In [ ]:
asr_model.

In [ ]:
from uberduck_ml_dev.text.symbols import SYMBOL_SETS, NVIDIA_TACO2_SYMBOLS

['_',
 '-',
 '!',
 "'",
 '(',
 ')',
 ',',
 '.',
 ':',
 ';',
 '?',
 ' ',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '@AA',
 '@AA0',
 '@AA1',
 '@AA2',
 '@AE',
 '@AE0',
 '@AE1',
 '@AE2',
 '@AH',
 '@AH0',
 '@AH1',
 '@AH2',
 '@AO',
 '@AO0',
 '@AO1',
 '@AO2',
 '@AW',
 '@AW0',
 '@AW1',
 '@AW2',
 '@AY',
 '@AY0',
 '@AY1',
 '@AY2',
 '@B',
 '@CH',
 '@D',
 '@DH',
 '@EH',
 '@EH0',
 '@EH1',
 '@EH2',
 '@ER',
 '@ER0',
 '@ER1',
 '@ER2',
 '@EY',
 '@EY0',
 '@EY1',
 '@EY2',
 '@F',
 '@G',
 '@HH',
 '@IH',
 '@IH0',
 '@IH1',
 '@IH2',
 '@IY',
 '@IY0',
 '@IY1',
 '@IY2',
 '@JH',
 '@K',
 '@L',
 '@M',
 '@N',
 '@NG',
 '@OW',
 '@OW0',
 '@OW1',
 '@OW2',
 '@OY',
 '@OY0',
 '@OY1',
 '@OY2',
 '@P',
 '@R',
 '@S',
 '@SH',
 '@T',
 '@TH',
 '@UH',
 '@UH0',
 '@U